In [51]:
import pandas as pd

# Read the log file
# You may need to adjust the path to your log file
log_file = "/Users/junzhouhe/expstuff/sampleWorkSpace/logs38"

try:
    with open(log_file, 'r') as f:
        logs = f.readlines()
    print(f"Successfully read {len(logs)} lines from log file")
except FileNotFoundError:
    print(f"Error: Could not find log file at {log_file}")
except Exception as e:
    print(f"Error reading log file: {str(e)}")


Successfully read 51018 lines from log file


In [52]:
class cmd_record:
    def __init__(self, cmd,timestamp,token):
        self.cmd = cmd
        self.start_timestamp = timestamp
        self.token = token



In [53]:
from pprint import pprint
cmd_records={}
for line in logs:
    if "received cmd:" in line:
        command_raw = line.split("received cmd:")[1].strip().split()[0]
        timestamp = line.split()[0]
        token = int(command_raw.split("-")[0])
        command = command_raw.split("-",1)[1]
        cmd_records[token] = cmd_record(command,timestamp,token)

for line in logs:
    if "output from token:" in line:
        token = int(line.split("output from token:")[1].strip().split()[0])
        timestamp = line.split()[0]
        if token in cmd_records:
            cmd_records[token].finish_timestamp = timestamp
# Filter out records without finish_timestamp
cmd_records = {token: record for token, record in cmd_records.items() 
              if hasattr(record, 'finish_timestamp')}


In [54]:
for token, cmd_record in cmd_records.items():
    print(cmd_record.cmd, cmd_record.start_timestamp, cmd_record.finish_timestamp)

bt-remote 2025-04-03T11:28:02.163257Z 2025-04-03T11:28:02.310969Z
bt-remote 2025-04-03T11:28:02.169149Z 2025-04-03T11:28:02.311750Z
bt-remote 2025-04-03T11:28:02.211895Z 2025-04-03T11:28:02.312036Z
bt-remote 2025-04-03T11:28:02.212065Z 2025-04-03T11:28:02.313147Z
bt-remote 2025-04-03T11:28:02.212237Z 2025-04-03T11:28:02.313759Z
bt-remote 2025-04-03T11:28:02.212401Z 2025-04-03T11:28:02.313164Z
bt-remote 2025-04-03T11:28:02.212588Z 2025-04-03T11:28:02.313966Z
bt-remote 2025-04-03T11:28:02.212770Z 2025-04-03T11:28:02.315400Z
bt-remote 2025-04-03T11:28:02.212933Z 2025-04-03T11:28:02.315456Z
bt-remote 2025-04-03T11:28:02.213105Z 2025-04-03T11:28:02.317400Z
bt-remote 2025-04-03T11:28:02.310997Z 2025-04-03T11:28:02.324646Z
bt-remote 2025-04-03T11:28:02.311786Z 2025-04-03T11:28:02.324476Z
bt-remote 2025-04-03T11:28:02.370315Z 2025-04-03T11:28:02.518443Z
bt-remote 2025-04-03T11:28:02.392177Z 2025-04-03T11:28:02.518950Z
bt-remote 2025-04-03T11:28:02.439875Z 2025-04-03T11:28:02.519355Z
bt-remote 

In [55]:
import datetime
import statistics
from pprint import pprint

def compute_latency_stats(latencies):
    """
    Given a list of numeric latency values (ms), compute:
      - count
      - mean
      - median
      - stddev (population standard deviation)
      - p95
      - p99
      - min
      - max

    Returns a dict with these keys. If empty, everything is 0.
    """
    if not latencies:
        return {
            'count': 0, 'mean': 0, 'median': 0, 'stddev': 0,
            'p95': 0, 'p99': 0, 'min': 0, 'max': 0
        }

    lat_sorted = sorted(latencies)
    count = len(latencies)
    mean_ = statistics.mean(latencies)
    median_ = statistics.median(latencies)
    stddev_ = statistics.pstdev(latencies)  # population std dev
    min_ = lat_sorted[0]
    max_ = lat_sorted[-1]

    idx_p95 = int(0.95 * (count - 1))
    idx_p99 = int(0.99 * (count - 1))
    p95 = lat_sorted[idx_p95]
    p99 = lat_sorted[idx_p99]

    return {
        'count': count,
        'mean': mean_,
        'median': median_,
        'stddev': stddev_,
        'p95': p95,
        'p99': p99,
        'min': min_,
        'max': max_
    }


def parse_iso8601(timestamp_str):
    """ Convert '2025-04-03T09:59:45.526590Z' to a datetime object. """
    # If your timestamps are strictly in ISO8601 format (with 'Z'), you can slice off the 'Z'
    # or use a more explicit parser with strptime. For simplicity:
    # 
    #   Example input: '2025-04-03T09:59:45.526590Z'
    #   We'll remove the trailing 'Z' and parse.
    #
    return datetime.datetime.fromisoformat(timestamp_str.replace("Z", ""))

# EXAMPLE USAGE:
def compute_all_cmd_stats(cmd_records):
    """
    1) Gather latencies in ms for each command (like 'bt-remote', 'exec-continue').
    2) Use compute_latency_stats() on each command's latencies.
    3) Return a dict: {cmd_name: stats_dict}
    """

    latencies_by_cmd = {}
    for token, record in cmd_records.items():
        # Make sure we have both start and finish timestamps
        if not hasattr(record, 'finish_timestamp'):
            continue
        
        start_dt = parse_iso8601(record.start_timestamp)
        finish_dt = parse_iso8601(record.finish_timestamp)
        diff_ms = (finish_dt - start_dt).total_seconds() * 1000.0

        latencies_by_cmd.setdefault(record.cmd, []).append(diff_ms)

    # For each command group, compute the stats
    stats_by_cmd = {}
    for cmd, lat_list in latencies_by_cmd.items():
        stats_by_cmd[cmd] = compute_latency_stats(lat_list)

    return stats_by_cmd

stats = compute_all_cmd_stats(cmd_records)


In [56]:
def print_stats_table(stats_by_cmd):
    """
    Expects a dict of the form:
      {
        "bt-remote": {
          'count': 12793, 'mean': 1924.1, 'median': 1433.6, 'stddev': 1817.0,
          'p95': 6877.8, 'p99': 7182.7, 'min': 11.8, 'max': 7374.1
        },
        "exec-continue": { ... },
        ...
      }

    Prints a nicely aligned ASCII table.
    """
    # Convert dict to a list of rows for printing
    rows = []
    for cmd_name, stats in stats_by_cmd.items():
        rows.append([
            cmd_name,
            stats['count'],
            f"{stats['mean']:.1f}",
            f"{stats['median']:.1f}",
            f"{stats['stddev']:.1f}",
            f"{stats['p95']:.1f}",
            f"{stats['p99']:.1f}",
            f"{stats['min']:.1f}",
            f"{stats['max']:.1f}"
        ])

    # Sort rows by command name (optional)
    rows.sort(key=lambda r: r[0])

    # Define table headers
    headers = ["Command", "Count", "Mean", "Median", "StdDev", "P95", "P99", "Min", "Max"]

    # Compute column widths
    col_widths = [len(h) for h in headers]
    for row in rows:
        for i, cell in enumerate(row):
            col_widths[i] = max(col_widths[i], len(str(cell)))

    # Helper to format a single row
    def fmt_line(cells):
        # For aesthetics: left-align first col (Command), right-align the rest
        line_parts = [
            str(cells[0]).ljust(col_widths[0])
        ]
        for i in range(1, len(cells)):
            line_parts.append(str(cells[i]).rjust(col_widths[i]))
        return "  ".join(line_parts)

    # Print title
    print("Statistical Analysis of Command Latencies (ms):\n")

    # Print header line
    header_line = fmt_line(headers)
    sep_line = "-" * len(header_line)
    print(header_line)
    print(sep_line)

    # Print each row
    for row in rows:
        print(fmt_line(row))
    print(sep_line)
print_stats_table(stats)

Statistical Analysis of Command Latencies (ms):

Command               Count  Mean  Median  StdDev    P95    P99   Min    Max
----------------------------------------------------------------------------
break-delete              1   1.8     1.8     0.0    1.8    1.8   1.8    1.8
break-insert              1  64.4    64.4     0.0   64.4   64.4  64.4   64.4
bt-remote              2348  32.5    14.2    39.2  128.5  153.9   4.5  275.1
exec-continue            37   2.2     2.0     0.6    3.0    4.2   1.8    4.5
exec-interrupt            4   8.1     7.3     2.7    7.8    7.8   5.3   12.5
stack-list-variables     66   3.1     2.6     2.0    4.6   12.8   1.6   13.1
var-create              146  24.4     7.4    20.7   48.5   51.2   1.6   65.0
var-list-children         3   3.0     3.0     0.2    3.0    3.0   2.8    3.2
var-update               11  19.2     4.6    20.2   47.6   47.6   2.4   47.6
----------------------------------------------------------------------------
